



# **IP2_Run1**

In [ ]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [ ]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))



### **Generate Mathematical Model**

In [ ]:
problem = """Your goal is to invest in several of 10 possible investment strategies in the most optimal way. The historic returns of those strategies are stored in the file "investments_data.csv". Each column represents one strategy and the rows are the past investment outcomes. There is no index and the values are separated by a ;.

The costs for investing in a given investment is stored in a vector A, which has one value for each strategy in order.
The values are: [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]

You can only invest once into an investment.

Unfortunately due to other costs and inflation, your available budget at this time is uncertain. There are four possible budget scenarios with different probabilities: scenario 1 with 1000 euros and probability of 0.55, scenario 2  with 1100 euros and probability of 0.4, scenario 3 with 900 euros and probability of 0.04, scenario 4 with 1200 euros and probability of 0.01.
The tolerable probability of exceeding the budget is 0.4.

Please formulate a mean-variance mathematical model for this optimization problem, considering the past performance of investment strategies and the uncertain budget. You can take 2 as the risk parameter r. """

In [ ]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [ ]:
print(response1.choices[0].message.content)

Decision Variables:
Let's denote binary variables xi, i ∈ {1, ..., 10}, which will take the value 1 if we decide to invest in the ith investment strategy, and 0 otherwise.

Parameters:
- mi: the mean return of the ith investment strategy, estimated from the historical data in the file.
- σi2: the variance of returns of the ith investment strategy, estimated from the historical data in the file.
- A: the cost for investing in a given strategy, which has a value for each strategy in order, denoted by Ai for ith strategy.
- Pj: probability of the jth budget scenario, j ∈ {1, 2, 3, 4}.
- Bj: available budget in the jth scenario, j ∈ {1, 2, 3, 4}.
- M: a positive big number.

Objective Function:
The objective is to maximize the expected return of the chosen investment strategies, penalizing high variability (risk). Therefore, our objective function will be to maximize:

Z = ∑mi*xi - r*∑σi2*xi 

for i from 1 to 10, where r is the given risk parameter (in this case, 2).

Constraints:

1. Tota

### **Generate Pyomo Code**

In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [ ]:
Markdown(response2.choices[0].message.content)

Here is the Python- Pyomo code for the given optimization problem:

```python
from pyomo.environ import *

# This is sample data, in actual code this should be replaced with original data
mean_returns = [12,15,17,11,13,16,10,14,13,14]
variance = [0.5, 1.5, 2.0, 1.0, 1.5, 1.5, 0.5, 1.0, 1.5, 1.5]
strategy_cost = [20,24,22,19,20,25,18,23,19,22]
prob_budget_scenarios = [0.2,0.7,0.9,0.1]
available_budget_scenarios = [200,170,230,185]
num_strategies = 10
budget_scenarios = 4
risk_parameter=2
M=10000

# Creating a model
model = ConcreteModel()

# Range of strategies as Index 
model.strategies = RangeSet(num_strategies)

#Range of budget scenarios as Index
model.scenarios=RangeSet(budget_scenarios)

#Defining decision variable
model.x = Var(model.strategies,within=Binary)

#Defining Objective function: Maximising mean returns - risk_prameter*variance
model.objective = Objective(expr = sum(mean_returns[i-1]*model.x[i] - risk_parameter*variance[i-1]*model.x[i] for i in model.strategies), 
                            sense=maximize)

#Defining the constraints for budget scenarios where probability >0.4
for j in model.scenarios:
    if prob_budget_scenarios[j-1]>0.4:
        model.add_component("budget_constraint_" + str(j),
                            Constraint(expr = sum(strategy_cost[i-1]*model.x[i] for i in model.strategies) <= 
                                       available_budget_scenarios[j-1]))
        
#Define Solver and solve
Solver = SolverFactory('glpk')
results = Solver.solve(model)

#Results
print('Objective = ', model.objective())
for i in model.strategies:
    print("Strategy ", i, " is ", model.x[i].value)
```

This code gives you the maximum expected return and the investment solution that tells you to invest or not to invest in each of the 10 strategies.

### **Run the code Generated by GPT4**

In [ ]:

from pyomo.environ import *

# This is sample data, in actual code this should be replaced with original data
mean_returns = [12,15,17,11,13,16,10,14,13,14]
variance = [0.5, 1.5, 2.0, 1.0, 1.5, 1.5, 0.5, 1.0, 1.5, 1.5]
strategy_cost = [20,24,22,19,20,25,18,23,19,22]
prob_budget_scenarios = [0.2,0.7,0.9,0.1]
available_budget_scenarios = [200,170,230,185]
num_strategies = 10
budget_scenarios = 4
risk_parameter=2
M=10000

# Creating a model
model = ConcreteModel()

# Range of strategies as Index
model.strategies = RangeSet(num_strategies)

# Range of budget scenarios as Index
model.scenarios=RangeSet(budget_scenarios)

# Defining decision variable
model.x = Var(model.strategies,within=Binary)

# Defining Objective function: Maximising mean returns - risk_prameter*variance
model.objective = Objective(expr = sum(mean_returns[i-1]*model.x[i] - risk_parameter*variance[i-1]*model.x[i] for i in model.strategies),
                            sense=maximize)

# Defining the constraints for budget scenarios where probability >0.4
for j in model.scenarios:
    if prob_budget_scenarios[j-1]>0.4:
        model.add_component("budget_constraint_" + str(j),
                            Constraint(expr = sum(strategy_cost[i-1]*model.x[i] for i in model.strategies) <=
                                       available_budget_scenarios[j-1]))

# Define Solver and solve
Solver = SolverFactory('glpk')
results = Solver.solve(model)

# Results
print('Objective = ', model.objective())
for i in model.strategies:
    print("Strategy ", i, " is ", model.x[i].value)

Objective =  89.0
Strategy  1  is  1.0
Strategy  2  is  0.0
Strategy  3  is  1.0
Strategy  4  is  0.0
Strategy  5  is  1.0
Strategy  6  is  1.0
Strategy  7  is  1.0
Strategy  8  is  1.0
Strategy  9  is  1.0
Strategy  10  is  1.0


In [ ]:
from pyomo.environ import *
import pandas as pd
# This is sample data, in actual code this should be replaced with original data

#start - data inputted by human
df = pd.read_csv("investments_data.csv",sep=";", header=None)
headers = []
for i in range(len(df.columns)):
    headers.append("Investment"+str((i+1)))
df.columns = headers

mean_returns = df.mean(0)
variance = df.var(0)
#end

strategy_cost = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]
prob_budget_scenarios = [0.55,0.4,0.04,0.01]
available_budget_scenarios = [1000,1100,900,1200]
num_strategies = 10
budget_scenarios = 4
risk_parameter=2
M=10000

# Creating a model
model = ConcreteModel()

# Range of strategies as Index
model.strategies = RangeSet(num_strategies)

# Range of budget scenarios as Index
model.scenarios=RangeSet(budget_scenarios)

# Defining decision variable
model.x = Var(model.strategies,within=Binary)

# Defining Objective function: Maximising mean returns - risk_prameter*variance
model.objective = Objective(expr = sum(mean_returns[i-1]*model.x[i] - risk_parameter*variance[i-1]*model.x[i] for i in model.strategies),
                            sense=maximize)

# Defining the constraints for budget scenarios where probability >0.4
for j in model.scenarios:
    if prob_budget_scenarios[j-1]>0.4:
        model.add_component("budget_constraint_" + str(j),
                            Constraint(expr = sum(strategy_cost[i-1]*model.x[i] for i in model.strategies) <=
                                       available_budget_scenarios[j-1]))

# Define Solver and solve
Solver = SolverFactory('glpk')
results = Solver.solve(model)

# Results
print('Objective = ', model.objective())
for i in model.strategies:
    print("Strategy ", i, " is ", model.x[i].value)


Objective =  225.96748959221745
Strategy  1  is  0.0
Strategy  2  is  0.0
Strategy  3  is  1.0
Strategy  4  is  0.0
Strategy  5  is  0.0
Strategy  6  is  1.0
Strategy  7  is  0.0
Strategy  8  is  1.0
Strategy  9  is  0.0
Strategy  10  is  0.0


In [ ]:
from pyomo.environ import *
import pandas as pd
# This is sample data, in actual code this should be replaced with original data

#start - data inputted by human
df = pd.read_csv("investments_data.csv",sep=";", header=None)
headers = []
for i in range(len(df.columns)):
    headers.append("Investment"+str((i+1)))
df.columns = headers

mean_returns = df.mean(0)
variance = df.var(0)
#end

strategy_cost = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]
prob_budget_scenarios = [0.55,0.4,0.04,0.01]
available_budget_scenarios = [1000,1100,900,1200]
num_strategies = 10
budget_scenarios = 4
risk_parameter=2
M=10000

# Creating a model
model = ConcreteModel()

# Range of strategies as Index
model.strategies = RangeSet(num_strategies)

# Range of budget scenarios as Index
model.scenarios=RangeSet(budget_scenarios)

# Defining decision variable
model.x = Var(model.strategies,within=Binary)

# Defining Objective function: Maximising mean returns - risk_prameter*variance
model.objective = Objective(expr = sum(mean_returns[i-1]*model.x[i] - risk_parameter*variance[i-1]*model.x[i] for i in model.strategies),
                            sense=maximize)

# Defining the constraints for budget scenarios where probability >0.4
for j in model.scenarios:
    if prob_budget_scenarios[j-1]>0.4:
        model.add_component("budget_constraint_" + str(j),
                            Constraint(expr = sum(strategy_cost[i-1]*model.x[i] for i in model.strategies) <=
                                       available_budget_scenarios[j-1]))

# Define Solver and solve
Solver = SolverFactory('glpk')
results = Solver.solve(model)

# Results
print('Objective = ', model.objective())
for i in model.strategies:
    print("Strategy ", i, " is ", model.x[i].value)


Objective =  225.96748959221745
Strategy  1  is  0.0
Strategy  2  is  0.0
Strategy  3  is  1.0
Strategy  4  is  0.0
Strategy  5  is  0.0
Strategy  6  is  1.0
Strategy  7  is  0.0
Strategy  8  is  1.0
Strategy  9  is  0.0
Strategy  10  is  0.0


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [ ]:
prob_budget_scenarios[j-1]

0.01